# Module 11. Advanced Machine Learning Models

Course: **TWEED Project, SC1. Libraries in Python**

Javier Lacasta, F. Javier López

- Exploration of classification models: logistic regression, decision trees, support vector machines.
- Model evaluation: performance metrics.
- Model selection and hyperparameter tuning.

## Environment Setup

- First, connect **Google Colab** to your **Google Drive** account.  
- We assume the course will be located in the root directory of your Drive and will be named `python-ml-course-TWEED`.  
- Inside this folder, there are two directories:
  - `data` containing the datasets used in the exercises.
  - `images` containing the images that will be visualized.

In [ ]:
try:
    import os
    from google.colab import drive
    drive.mount('/content/drive')
    ROOT = '/content/drive/MyDrive/python-ml-course-TWEED/'
except ImportError:
    ROOT = './'
    pass

## Scikit Learn library

**Scikit-Learn** is a reference library for **classical machine learning** tasks such as classification, regression, clustering, and dimensionality reduction. It provides a wide range of efficient algorithms built on top of NumPy, SciPy, and Matplotlib. It provides an **homogeneous API design**, where all models share the same basic methods — `fit()`and `predict()` — allowing to switch between algorithms or combine them in pipelines with minimal code changes.

| Type | Main Class  |
|------|-------------|
| Regression | `LinearRegression`, `RandomForestRegressor`|
| Classification | `LogisticRegression`, `RandomForestClassifier`, `SVC` |
| Clustering | `KMeans`, `DBSCAN` |
| Dimensionality Reduction | `PCA`, `t-SNE` |

### Linear Regression (Prediction)

- **Description:** Model to predict a continuous variable based on one or more predictor variables.
- **Formula:** $Y=\beta_0+\beta_1X_1+\ldots+\beta_mX_m+\epsilon$
- **Goal:** Predict values with a linear behavior.
- **Optimization:** Find coefficients $\beta$ that minimize the error.
  - **Main:** Mean Squared Error (MSE) - $MSE = \frac{\sum (y_i-y_i^p)^2}{n}$
  - **Alternatives:** Mean Absolute Error, Log-Cosh of the error.
- **More Information:** Visit [scikit-learn](https://scikit-learn.org/stable/modules/linear_model.html) for linear models.

## Application Example: Linear Regression in Gas Sensor

- **Context:** Sensor calibration to measure gas concentrations.
- **Behavior:** Different gases produce different signal intensities, allowing identification of concentration.
- **Challenge:** Interaction between multiple gases affects the sensor response.
- **Goal:** Develop a model that predicts sensor intensities for various gas mixtures.

### Data Transformation

In [ ]:
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt

data = pd.read_excel(ROOT + 'data/gases.xlsx', sheet_name='Concentracion').astype(float)
data.head(5)

- **Attention to Vector Dimensions**
  - **Important:** Different functions vary in how they require input vectors.
  - **Recommendation:** Check the specific documentation to understand the correct input format.

In [ ]:
print(data["Conc Ar"].to_numpy()[0:5])
print(data["Conc Ar"].to_numpy()[0:5].reshape(-1, 1))

### Sensor Behavior with Respect to Argon

- Nearly linear relationship between signal intensity and Argon concentration
- **Seaborn** allows visualization of regressions but does not provide access to the coefficients

In [ ]:
sns.regplot(x="Conc Ar", y="Int Ar", data=data)

- **Tool**: Use **sklearn** to define regression coefficients.
- **Error Metric**: Mean Squared Error.
- **Advantage**: Mathematical solution without the need to scale values.
- **Coefficient of Determination**: Measures the proportion of variation in the dependent variable explained by the independent variable.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np

x = data["Conc Ar"].to_numpy().reshape(-1, 1)
y = data["Int Ar"].to_numpy().reshape(-1, 1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
reg = LinearRegression().fit(x_train, y_train)

print(f"Coefficient of determination (Train): {reg.score(x_train, y_train):.4f}")
print(f"Coefficient of determination (Test):  {reg.score(x_test, y_test):.4f}")
print(f"Model Coefficients: {reg.coef_[0]}")
print(f"Model Intercept: {reg.intercept_}")

- The model allows predicting intensity given concentration

In [ ]:
y1 = reg.predict(x)
plt.figure(figsize=(10, 6))
sns.regplot(x="Conc Ar", y="Int Ar", data=data)
plt.plot(x, y1, 'o', color='black');

### Sensor Behavior with Respect to Nitrogen

- The intensity generated by Nitrogen is affected by the concentration of other gases.

In [ ]:
sns.jointplot(x=data["Conc N2"], y=data["Int N2"],  kind='reg')

- The linear model is not suitable because the coefficient of determination is much lower

In [ ]:
x = data["Conc N2"].to_numpy().reshape(-1, 1)
y = data["Int N2"].to_numpy().reshape(-1, 1)
reg = LinearRegression().fit(x, y)
print("Coefficient of determination:", reg.score(x, y))

### Creation of a Multivariable Linear Regression Model

- The presence of other gases must be taken into account in the regression

In [ ]:
x = data[["Conc H2", "Conc Ar", "Conc N2", "Conc CH4", "Conc CO2", "Conc CO"]].to_numpy().reshape(-1, 6)
y = data["Int N2"].to_numpy().reshape(-1, 1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
reg = LinearRegression().fit(x_train, y_train)

print(f"Coefficient of determination (Train): {reg.score(x_train, y_train):.4f}")
print(f"Coefficient of determination (Test):  {reg.score(x_test, y_test):.4f}")
print(f"Model Coefficients: {reg.coef_[0]}")
print(f"Model Intercept: {reg.intercept_}")
reg.predict(np.array([10, 10, 10, 10, 10, 10]).reshape(1, -1))

In [ ]:
y = reg.predict(x)
plta = data.plot.scatter(x='Conc N2', y='Int N2', c='red')
plta.plot(data["Conc N2"], y, 'o', color='black');

- **Caution:** Fitting the training data well does not guarantee a good model.
- **Risk of Overfitting:** The model may fail to predict new data.
- **Effective Validation:** Split data into training set (70-80%) and test set (20-30%)

### Logistic Regression

- **Purpose:** Predict categorical variables from predictor variables.  
- **Use Case:** Classification into a limited number of categories.  
- **Sigmoid Function:** Used to model the relationship between predictor variables and the categorical variable.  
  - $h = \frac{1}{1+e^{-x}}$ with $x = b_0 + b_1x_1 + b_2x_2 + ... + b_nx_n$.  
- **Error Measurement:** Cost function penalizing deviations from the correct prediction.  
- **Cost Function:** Different for $y = 1$ and $y = 0$, based on the logarithm of the predicted probability.  
  - $Cost(h_\theta(x),y)=
\begin{cases}
-\log(h_\theta(x)) && \text{if }y = 1\\
-\log(1-h_\theta(x)) && \text{if }y = 0
\end{cases}$  
- **Visual Example:** Considering a single feature ($x = b_0 + b_1x_1$), we would have an image like:

In [ ]:
from IPython.display import Image, display

display(Image(ROOT + 'images/logisticregresionerror.png', width=300))

- **More Information:** Visit scikit-learn’s page on [Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).

#### Logistic Regression Application: Star Classification

- **Experimental Data:** Features of stars such as temperature, luminosity, etc.  
- **Objective:** Develop a model that classifies stars into 6 different types using logistic regression to handle the predictor variables.

In [ ]:
import pandas as pd

data = pd.read_excel(ROOT + 'data/stars.xlsx', skiprows=1)
data.head(5)

- There are forty stars of each type

In [ ]:
data['Type'].value_counts()

- We transform the data into numeric types

In [ ]:
data['Color'] = pd.factorize(data['Color'])[0] 
data['Clase Espectral'] = pd.factorize(data['Clase Espectral'])[0]
data.describe()

#### Building a Logistic Regression Model

- **Fitting Method:** Uses gradient descent to optimize parameters.  
- **Data Preparation:** Requires scaling variables to a common range.  
- **Training Options:** Several optimization strategies available (`newton-cg`, `lbfgs`, `liblinear`, `sag`, `saga`).  
- **Working Modes:**  
  - **OVR (One-vs-Rest):** Creates one binary classifier per category.  
  - **Multinomial:** A single classifier handles all categories simultaneously.  

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

x = data[["Temperature", "Luminosidad", "Radio", "Magnitud Absoluta", "Color", "Clase Espectral"]].to_numpy()
y = data["Type"].to_numpy()
scaledX = MinMaxScaler().fit_transform(x)
X_train, X_test, y_train, y_test = train_test_split( scaledX, y, test_size=0.3, random_state=42, stratify=y)  # Maintain class distribution

clf = LogisticRegression(solver='lbfgs', max_iter=1000).fit(X_train, y_train)
print(f"Accuracy TRAIN: {clf.score(X_train, y_train):4.2f}")
print(f"Accuracy TEST:  {clf.score(X_test, y_test):4.2f}")

y_pred_test = clf.predict(X_test)
plt.figure(figsize=(6,4))
cm_test = confusion_matrix(y_test, y_pred_test)
sns.heatmap(cm_test, annot=True,  annot_kws={"size": 16})
plt.title('Confusion Matrix - TEST Data', fontsize=16)
plt.ylabel('True Value')
plt.xlabel('Prediction')
plt.show()

print(classification_report(y_test, y_pred_test))


### Decision Trees

- **Purpose:** Represent decisions and their possible outcomes sequentially.  
- **Characteristics:** Each node defines a condition and the associated error.  
- **Computation:** Uses the Gini index to evaluate and minimize impurity in data splits.  
  - **Gini Formula:** $Gini\,Impurity = 1 - (Probability\,of\,Class\,1)^2 - (Probability\,of\,Class\,2)^2$  
- **Optimization:** Searches for splits that minimize the Gini index.  
- **Random Forest (Generalization):** Combines multiple decision trees to improve prediction stability and accuracy.  
- **More Information:** Visit [scikit-learn for Decision Trees and Random Forests](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.tree).

#### Model Training

- Since the optimization algorithm requires it, variable scaling is necessary.  This algorithm also requires separation between training and test.

In [ ]:
from sklearn import tree
from sklearn.utils import shuffle

scaledX, y = shuffle(scaledX, y, random_state=0)
clf = tree.DecisionTreeClassifier(random_state=0)
clf = clf.fit(scaledX[:168, :], y[:168])
testX = scaledX[168:, :]
testY = y[168:]
y1 = clf.predict(testX)
clf.score(testX, testY)

clf = tree.DecisionTreeClassifier(random_state=0).fit(X_train, y_train)
train_score = clf.score(X_train, y_train)
test_score = clf.score(X_test, y_test)

print(f"Accuracy TRAIN: {clf.score(X_train, y_train):4.2f}")
print(f"Accuracy TEST:  {clf.score(X_test, y_test):4.2f}")

y_pred_test = clf.predict(X_test)
plt.figure(figsize=(6,4))
cm_test = confusion_matrix(y_test, y_pred_test)
sns.heatmap(cm_test, annot=True,  annot_kws={"size": 16})
plt.title('Confusion Matrix - TEST Data', fontsize=16)
plt.ylabel('True Value')
plt.xlabel('Prediction')
plt.show()



### Other models are used in exactly the same way

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn import svm

gnb = GaussianNB().fit(X_train, y_train)
print(gnb.score(testX, testY))
y1 = gnb.predict(testX)

svm = svm.SVC(kernel='rbf').fit(X_train, y_train)
print(svm.score(testX, testY))
y1 = svm.predict(testX)

## Model Evaluation: Performance Metrics

- How good is a model?
- Let's analyze a simple case using an SVR model.
- SVR is similar to SVM, but is used in regression problems.

### Test Data

- Let's continue working with the pig price history series.

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_excel(ROOT + 'data/prices_clean.xlsx', sheet_name='Selecto', index_col=0)
data.head(5)

- The data consists of the evolution of prices in different markets over 5 years

In [ ]:
data["Lerida"].plot(kind='line', figsize=(10, 5))

- If we didn't have the values ​​for Lérida and had to predict them, given the existing correlation, we could train a model to do so.

In [ ]:
import matplotlib.pyplot as plt, seaborn as sns

sns.set(rc={'figure.figsize': (10, 10)})
sns.heatmap(abs(data.corr()), vmin=0, vmax=1, center=0, square=True)

### Preparation

- We focused on market data from Spain, as it is most correlated with Lérida.

In [ ]:
x = data.drop(['Lerida', 'Spain', 'Portugal', 'Germany', 'Netherlands',
               'Austria', 'France', 'Hungary', 'Poland', 'Denmark', 'Croatia',
               'Slovenia', 'Ireland', 'Estonia', 'Belgium', 'Lihtuania', 'Czechia',
               'Slovakia', 'Latvia', 'Romania', 'Bulgaria', 'Cyprus', 'Greece',
               'Finland'], axis=1)
y = pd.DataFrame(data["Lerida"]).set_index(data.index)

- We separate the data into training and test data
- In time series, the oldest data is used as training data and the most recent data is used as test data

In [ ]:
limit = int(len(x) * 0.8)
X_train = x[:limit]
X_test = x[limit:]
Y_train = y[:limit]
Y_test = y[limit:]
dates_train = X_train.index
dates_test = X_test.index

- We make the experiment replicable by setting a fixed seed in the random generators

In [ ]:
import random as python_random

np.random.seed(0)
python_random.seed(0)

### Default Prediction with SVR

- **Goal**: Obtain a baseline for parameter optimization
- **Task**: Train the model and predict with the default parameters:
- **Kernel**: `rbf` (Radial Basis Function)
- **Degree**: 3, polynomial degree for `poly` functions (ignored by `rbf`)
- **Gamma**: `scale=1`, coefficient of the `rbf`, `poly`, and `sigmoid` kernels.
- **C**: 1.0, regularization parameter. Used to avoid overfitting.
- **Epsilon**: 0.1, unpenalized error margin during training

In [ ]:
from sklearn.svm import SVR

model = SVR().fit(X_train, Y_train['Lerida'].to_numpy())
Y_pred = model.predict(X_test)

### Cross-Validation

- **Problem**: Separating training and test data can influence results if they are not representative of the entire value space.
- **Solution**: Cross-validation involves performing multiple experiments, changing the test subset.
- **Model Error**: The average error of the predictions of the different trained models.

In [ ]:
from IPython.display import Image, display

display(Image(ROOT + 'images/K-fold_cross_validation.jpg', width=400))

In [ ]:
from sklearn.model_selection import cross_val_score

pred = cross_val_score(model, x, y['Lerida'].to_numpy(), cv=5)
pred

### Measuring the quality of the results

- To measure the quality of the results and compare them with the optimized version, we will use a set of error measures.
- **Mean square error**
- **Root of mean square error**
- **Mean absolute error**
- **$R^2$ Score**: Coefficient of determination
- **Cross Validation Score**: Default error measure of the model to be trained (usually $R^2$)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error


def cross_val(model):
    pred = cross_val_score(model, x, y['Lerida'].to_numpy(), cv=5)
    return pred.mean()


def metrics(valid, pred):
    mse = mean_squared_error(valid, pred)
    rmse = np.sqrt(mean_squared_error(valid, pred))
    mae = mean_absolute_error(valid, pred)
    r2 = r2_score(valid, pred)
    return mse, rmse, mae, r2


df_models_temp = pd.DataFrame(data=[['SVR', *metrics(Y_test['Lerida'].to_numpy(), Y_pred), cross_val(SVR())]],
                              columns=['Model Name', 'MSE', 'RMSE', 'MAE', "r2", 'Cross Val'])
df_models_temp.head(1)

- We can see that it is not a good result

In [ ]:
import matplotlib.dates as mdates


def grafico(dates_test, y_pred, y_test):
    plt.figure(figsize=(17, 10))
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%d/%Y'))
    plt.plot(dates_test, y_pred, c='red', label='Predict', marker='o', linestyle='-',
             markersize=10, markeredgecolor='red', markerfacecolor='red', alpha=0.5)  #alpha = transparency
    plt.plot(dates_test, y_test, c='black', label='Real', marker='D', linestyle='--', markersize=8,
             markeredgecolor='black', markerfacecolor='black', alpha=0.5)
    plt.legend(loc='lower right', fontsize=22)
    plt.xlabel('Date', fontsize=20)
    plt.ylabel('Price', fontsize=20)
    plt.title('SVR predictions', fontsize=25)


grafico(dates_test, Y_pred, Y_test['Lerida'].to_numpy())
plt.show()

## Model Selection and Hyperparameter Tuning

### Model Diagnosis

- The model has not been tuned adequately.
- Possible causes:
- The selected model is not suitable for the problem.
- The model's hyperparameters are incorrectly configured.

### Solution: Hyperparameter Tuning

- Perform a hyperparameter sweep:
- Select a wide range of values ​​for each parameter.
- Use Scikit-Learn's **GridSearchCV** to explore all possible combinations.
- Benefits of **GridSearchCV**
- Automates the search for the best parameter combination.
- Presents the best result obtained from the tested combinations.
- Problems with **GridSearchCV**
- How to define a grid that covers the desired optimum?
- How to prevent it from being computationally too large?

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel': ('rbf', 'linear'), 'C': [2, 2.5, 3], 'epsilon': [0.01, 0.5, 1], 'gamma': ('auto', 'scale'),
              'degree': [3, 4, 5]}
clf = GridSearchCV(SVR(), parameters)
clf.fit(X_test, Y_test['Lerida'].to_numpy())
best_params = clf.best_params_
best_params

- We use the parameters to train the model and compare the results

In [ ]:
model = SVR(kernel=best_params["kernel"], C=best_params["C"], gamma=best_params["gamma"],
            epsilon=best_params["epsilon"], degree=best_params["degree"]).fit(X_train, Y_train['Lerida'].to_numpy())
Y_pred = model.predict(X_test)

df_modelsTmp2 = pd.DataFrame(data=[['SVR + GridSearchCV', *metrics(Y_test['Lerida'].to_numpy(), Y_pred), cross_val(
    SVR(kernel=best_params["kernel"], C=best_params["C"], gamma=best_params["gamma"], epsilon=best_params["epsilon"],
        degree=best_params["degree"]))]], columns=['Model Name', 'MSE', 'RMSE', 'MAE', "r2", 'Cross Val'])
df_modelsTmp2 = pd.concat([df_modelsTmp2, df_models_temp], ignore_index=True)
df_modelsTmp2

- Let's see the result of the optimized model

In [ ]:
grafico(dates_test, Y_pred, Y_test['Lerida'].to_numpy())
plt.title('SVR predictions', fontsize=25)
plt.show()

### Parameter Search Optimization

- Use of optimization libraries such as **Optuna**:
- Does not test all combinations; uses optimization algorithms
- Finds optimal parameter values ​​by minimizing error
- Advantage of **Optuna**:
- Significantly reduces the time required to find the best configuration.

In [ ]:
! pip install optuna

- **Optuna** requires an objective function (`objective`) with:
- The parameter value ranges
- The model to be trained
- The quality/error value to be maximized/minimized
- Categorical parameters are fully analyzed
- Numerical parameters are tested against previous results

In [ ]:
import optuna as optuna

def objective(trial):
    kernel = trial.suggest_categorical('kernel', ['rbf', 'poly', 'linear','sigmoid'])
    c = trial.suggest_float("C", 2, 3, log=True)
    epsilon = trial.suggest_float("epsilon", 0.00001, 1, log=True)
    gamma = trial.suggest_categorical('gamma', ['auto', 'scale'])
    
    if kernel == 'poly': degree = trial.suggest_int("degree", 2, 5)
    else: degree = 3  # Default degree for non-poly kernels
    
    model = SVR(kernel=kernel, degree=degree, gamma=gamma, C=c, epsilon=epsilon)
    scores = cross_val_score(model, X_train, Y_train.values.reshape(-1), cv=5, scoring='r2')
    return scores.mean() - 0.1 * scores.std()  # Penalize high variance between folds


study = optuna.create_study(direction="maximize", sampler=optuna.samplers.RandomSampler(seed=123))
study.optimize(objective, n_trials=5)
trial = study.best_trial
print("Best parameters : {} \n with accuracy of : {:.2f}".format(trial.params, trial.value))
best_params = dict(trial.params)

- Use the parameters to train the model and compare results

In [ ]:
model = SVR(kernel=best_params["kernel"], C=best_params["C"], gamma=best_params["gamma"],
            epsilon=best_params["epsilon"]).fit(X_train, Y_train['Lerida'].to_numpy())
Y_pred = model.predict(X_test)

df_modelsTmp3 = pd.DataFrame(data=[['SVR + Optuna', *metrics(Y_test['Lerida'].to_numpy(), Y_pred), cross_val(
    SVR(kernel=best_params["kernel"], C=best_params["C"], gamma=best_params["gamma"], epsilon=best_params["epsilon"]
        ))]], columns=['Model Name', 'MSE', 'RMSE', 'MAE', "r2", 'Cross Val'])
df_modelsTmp3 = pd.concat([df_modelsTmp3, df_modelsTmp2], ignore_index=True)
df_modelsTmp3

- Let's see the result of the optimized model

In [ ]:
grafico(dates_test, Y_pred, Y_test['Lerida'].to_numpy())
plt.title('SVR predictions', fontsize=25)
plt.show()

### Hyperparameter Contribution Analysis

- **Objective**: Determine the importance of each parameter in optimizing results.
- **Process**:
- Identify the most relevant parameters for model prediction.
- Evaluate how each parameter influences model performance.

### Focused Optimization Strategy

- **Steps to follow**:
- Set the least relevant parameters to previously identified optimal values.
- Concentrate optimization efforts on key parameters that have the greatest impact on model improvement.
- **Benefits of Focused Optimization**
- Optimizes resources by focusing attention on areas of greatest impact.
- Potentially improves model performance more significantly by adjusting the most influential parameters.

In [ ]:
plt.rcParams["figure.figsize"] = (10, 6)
optuna.visualization.matplotlib.plot_param_importances(study)

## End of session

In [ ]:
try:
    from google.colab import drive

    drive.flush_and_unmount()
except ImportError:
    pass